In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv("../data/processed/train_df_fold.csv")
# とりあえずfoldは落とす
df = df.drop("fold", axis=1)
df

,epoch,meas_time,label,id
0,0,1989-11-13 21:35:00,0,3c1c5cf
1,1,1989-11-13 21:35:30,0,3c1c5cf
2,2,1989-11-13 21:36:00,0,3c1c5cf
3,3,1989-11-13 21:36:30,0,3c1c5cf
4,4,1989-11-13 21:37:00,0,3c1c5cf
...,...,...,...,...
161605,1475,1990-03-14 08:17:30,1,5edb9d9
161606,1476,1990-03-14 08:18:00,1,5edb9d9
161607,1477,1990-03-14 08:18:30,2,5edb9d9
161608,1478,1990-03-14 08:19:00,2,5edb9d9


In [3]:
npy_sample = np.load("../data/0aa8d07_0.npy")
npy_sample.shape

(3000, 7)

id は一意なので id で groupby して連続を探す

In [4]:
ids = df["id"].unique()

tmp = df[df["id"] == ids[0]]
tmp = tmp[tmp["label"] == 1]

epochs = tmp["epoch"].values
meas_times = tmp["meas_time"].values

In [5]:
print(type(meas_times[0]))
print(meas_times[0])

# datetime型に変換
meas_times = [pd.to_datetime(meas_time) for meas_time in meas_times]
print(type(meas_times[0]))
print(meas_times[0])

<class 'str'>
1989-11-13 21:40:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
1989-11-13 21:40:00


In [6]:
# idを抽出
ids = df["id"].unique()
print(len(ids))

108


In [7]:
# 共通するidごとにepochが連続するものを抽出
for i in tqdm(ids[:10]):
    # labelが1のものを抽出
    tmp = df[df["id"] == i]
    tmp = tmp[tmp["label"] == 1]
    # epochの値が2つ以上連続
    epochs = tmp["epoch"].values
    # int型に変換
    epochs = [int(epoch) for epoch in epochs]
    meas_times = tmp["meas_time"].values
    # datetime型に変換
    meas_times = [pd.to_datetime(meas_time) for meas_time in meas_times]
    epoch_prev = epochs[0]
    for (epoch, meas_time) in zip(epochs[1:], meas_times[:-1]):
        if epoch - epoch_prev == 1:
            # 読み込み、新規ファイル生成
            seq_prev = np.load(f"../data/arrays/{i}_{epoch_prev}.npy")
            seq_curr = np.load(f"../data/arrays/{i}_{epoch}.npy")
            seq = np.concatenate([seq_prev, seq_curr])
            for j in range(1, 3):
                idx_start = 1000 * j
                seq_dst = seq[idx_start:idx_start + 3000]
                fname = f"../data/arrays/{i}_{epoch_prev}_{j}.npy"
                df_append = pd.DataFrame(
                    {
                        "epoch": [f"{epoch_prev}_{j}"],
                        "meas_time": [meas_time + pd.Timedelta(seconds=10 * j)],
                        "label": [1],
                        "id": [i],
                    }
                )
                df = pd.concat([df, df_append])
        epoch_prev = epoch

100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


In [8]:
print(seq_dst.shape)

(3000, 7)


In [9]:
df[df["id"] == ids[0]]

,epoch,meas_time,label,id
0,0,1989-11-13 21:35:00,0,3c1c5cf
1,1,1989-11-13 21:35:30,0,3c1c5cf
2,2,1989-11-13 21:36:00,0,3c1c5cf
3,3,1989-11-13 21:36:30,0,3c1c5cf
4,4,1989-11-13 21:37:00,0,3c1c5cf
...,...,...,...,...
0,1415_2,1989-11-14 09:22:50,1,3c1c5cf
0,1422_1,1989-11-14 09:26:10,1,3c1c5cf
0,1422_2,1989-11-14 09:26:20,1,3c1c5cf
0,1428_1,1989-11-14 09:29:10,1,3c1c5cf


In [11]:
df[(df["label"] == 3) & (df["id"] == ids[0])]

,epoch,meas_time,label,id
821,841,1989-11-14 04:35:30,3,3c1c5cf
987,1007,1989-11-14 05:58:30,3,3c1c5cf
988,1008,1989-11-14 05:59:00,3,3c1c5cf
989,1009,1989-11-14 05:59:30,3,3c1c5cf
990,1010,1989-11-14 06:00:00,3,3c1c5cf
992,1012,1989-11-14 06:01:00,3,3c1c5cf
993,1013,1989-11-14 06:01:30,3,3c1c5cf
995,1015,1989-11-14 06:02:30,3,3c1c5cf
996,1016,1989-11-14 06:03:00,3,3c1c5cf
997,1017,1989-11-14 06:03:30,3,3c1c5cf


In [29]:
data_sample = [
    ["0",   "2020-01-01 00:00:00", 1, "a", 0],
    ["1",   "2020-01-01 00:00:00", 1, "b", 0],
    ["0_1", "2020-01-01 00:00:10", 1, "a", 0],
    ["0_2", "2020-01-01 00:00:20", 1, "a", 0],
    ["1_1", "2020-01-01 00:00:10", 1, "b", 0],
]

df_sample = pd.DataFrame(data_sample, columns=["epoch", "meas_time", "label", "id", "fold"])

In [30]:
# これはすべきかわからない
epochs_test = df_sample["epoch"].values.astype(str)
epochs_test

array(['0', '1', '0_1', '0_2', '1_1'], dtype='<U3')

In [34]:
df_sample["epoch_formatted"] = df_sample["epoch"].str.split("_").str[0].astype(int)
d = df_sample.groupby("id").apply(lambda x: x)
d

,epoch,meas_time,label,id,fold,epoch_formatted
0,0,2020-01-01 00:00:00,1,a,0,0
1,1,2020-01-01 00:00:00,1,b,0,1
2,0_1,2020-01-01 00:00:10,1,a,0,0
3,0_2,2020-01-01 00:00:20,1,a,0,0
4,1_1,2020-01-01 00:00:10,1,b,0,1
